In [ ]:
# pip install requests bs4 lxml xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 2.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from requests.compat import urljoin
from bs4 import BeautifulSoup
import re
import xlsxwriter
import time
import csv

In [ ]:
def CleanData(InfoString):
    InfoString = InfoString.replace('\n', '',)  
    InfoString = InfoString.replace('\r', '',)  
    InfoString = InfoString.replace(' ', '',)
    InfoString = InfoString.replace('年', '-', 1)  
    InfoString = InfoString.replace('月', '-', 1)
    InfoString = InfoString.replace('日', '', 1)
    return InfoString

def ExtractBJWeather():
    BJWeatherExcel = xlsxwriter.Workbook('北京天气爬虫.xlsx')
    url = 'http://www.tianqihoubao.com/lishi/beijing.html'
    head = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36','Connection': 'close'}
    html = requests.get(url, headers = head)
    bsObj = BeautifulSoup(html.content, 'lxml') 

    OneYearWeather = bsObj.find_all('div', class_ = "box pcity")  
    Year = 2010
    for AQuarter in OneYearWeather:  
        Year += 1
        if Year <= 2020: 
            WorkSheet = BJWeatherExcel.add_worksheet(str(Year))
            row = 0
            col = 0
            WorkSheet.write(row, col, '日期')  
            WorkSheet.write(row, col+1, '天气状况')
            WorkSheet.write(row, col+2, '气温')
            WorkSheet.write(row, col+3, '风力风向')       
            AQuarterData = AQuarter.find_all('ul')  
            for AMonth in AQuarterData:  # 一个季度内遍历每个月的数据
                # print(AMonth)  
                ThreeMonthLink = AMonth.find_all('a')  
                for Link in ThreeMonthLink: 
                    AMonthLink = Link['href']
                    if '/lish' in str(AMonthLink): 
                        AMonthurl = 'http://www.tianqihoubao.com' + AMonthLink  
                    else: 
                        AMonthurl = 'http://www.tianqihoubao.com/lish' + AMonthLink
                    try: 
                        AMonth_HTML = requests.get(AMonthurl, headers = head) 
                        AMonthObj = BeautifulSoup(AMonth_HTML.content, 'lxml')  
                        AMonthData = AMonthObj.find_all('tr') 
                        i = 1
                        for ADay in AMonthData: 
                            if i ==1:
                                i = 2
                                continue
                            else:
                                ALine = ADay.find_all('td') 
                                col = 0 
                                row += 1
                                for info in ALine: # 遍历行的四项信息
                                    AnInfo = str(info.get_text())  # 获取行数据下的文本内容
                                    AnInfo = CleanData(AnInfo) # 除去数据中的\n、\r、空格，以及变换日期格式
                                    WorkSheet.write(row, col, AnInfo) # 数据写入表格
                                    col += 1 # 列增加
                    except:  # 请求过快时，按5s休息处理
                        print("requests speed so high,need sleep!")
                        time.sleep(5)
                        print("continue...")
                        continue
            print('...爬完 '+ str(Year) + ' 年...')
        else:
            break
    BJWeatherExcel.close()
    print('---------------------end----------------')
    return 

if __name__ == '__main__':
    ExtractBJWeather()   

...爬完 2011 年...
...爬完 2012 年...
...爬完 2013 年...
requests speed so high,need sleep!
continue...


In [4]:
def CleanData(InfoString):
    InfoString = InfoString.replace('\n', '',)  
    InfoString = InfoString.replace('\r', '',)  
    InfoString = InfoString.replace(' ', '',)
    InfoString = InfoString.replace('年', '-', 1)  
    InfoString = InfoString.replace('月', '-', 1)
    InfoString = InfoString.replace('日', '', 1)
    return InfoString

def ExtractBJWeather():
    # Scrape Beijing weather data
    output_file = 'Beijing_Weather_Data.csv'
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Date', 'Weather Condition', 'Temperature', 'Wind Force and Direction'])  # CSV headers

        url = 'http://www.tianqihoubao.com/lishi/beijing.html'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
            'Connection': 'close'
        }
        html = requests.get(url, headers=headers)
        html.encoding = 'gb2312'  # Correctly set the encoding to match the website
        bsObj = BeautifulSoup(html.text, 'lxml')  # Use text instead of content with the correct encoding

        year_weather_data = bsObj.find_all('div', class_="box pcity")  # Weather data for all years
        year = 2010
        for quarter_data in year_weather_data:  # Iterate through quarterly data
            year += 1
            if year <= 2011:  # Restrict to data up to 2020
                quarter_list = quarter_data.find_all('ul')  # Quarterly data
                for month_data in quarter_list:  # Iterate through months in a quarter
                    month_links = month_data.find_all('a')  # Extract links for each month
                    for link in month_links:
                        month_url = urljoin('http://www.tianqihoubao.com', link['href'])
                        try:
                            month_html = requests.get(month_url, headers=headers)
                            month_html.encoding = 'gb2312'  # Correct encoding
                            month_bsObj = BeautifulSoup(month_html.text, 'lxml')
                            month_rows = month_bsObj.find_all('tr')  # Rows of monthly data
                            for i, day_row in enumerate(month_rows):
                                if i == 0:  # Skip the header row
                                    continue
                                day_data = [CleanData(td.get_text()) for td in day_row.find_all('td')]
                                if len(day_data) == 4:
                                    writer.writerow(day_data)  # Write data to CSV file
                        except Exception as e:  # Handle request rate limit
                            print("Request rate too high, sleeping for 5 seconds...")
                            time.sleep(5)
                            print("Retrying...")
                            continue
                print(f'Finished scraping data for {year}...')
            else:
                break
    print('---------------------Scraping Complete----------------')
    return


if __name__ == '__main__':
    ExtractBJWeather()   

Finished scraping data for 2011...
---------------------Scraping Complete----------------
